# Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls /content/gdrive/My\ Drive

Mounted at /content/gdrive
 aimas2020
'Automatic Generation of Topic Labels.gslides'
'Colab Notebooks'
 cvdl2020
 iir_book.pdf
 ir_final
'Medical AI'
'Paper Slides'
 Q56094077
 res18_diabete_noaug.pth
'Towards Better Text Understanding and Retrieval through Kernel Entity Saliency Modeling.gslides'
 tsai.ipynb
 獎助學金
 申請資料


In [ ]:
# !unzip /content/gdrive/MyDrive/Q56094077/snrs/hw1_0319/hw1_data.zip -d /content/gdrive/MyDrive/Q56094077/snrs/hw1_0319

# Import Library

In [20]:
import os

import torch
import torch.nn as nn

import pandas as pd
import numpy as np

from tqdm import tqdm

In [7]:
import torch_geometric
from torch_geometric.data import Data, DataLoader
import torch_geometric.utils as utils

# Define constant

In [8]:
_root = os.getcwd()

_data = os.path.join(_root, "hw1_data")

data_synthetic = os.path.join(_data, "Synthetic", "5000")
data_youtube = os.path.join(_data, "youtube")

# Dataset

## Data

- data.x	节点特征，维度是[num_nodes, num_node_features]。
- data.edge_index	维度是[2, num_edges]，描述图中节点的关联关系，每一列对应的两个元素，分别是边的起点和重点。数据类型是torch.long。需要注意的是，data.edge_index是定义边的节点的张量（tensor），而不是节点的列表（list）。
- data.edge_attr	边的特征矩阵，维度是[num_edges, num_edge_features]
- data.y	训练目标（维度可以是任意的）。对于节点相关的任务，维度为[num_nodes, *]；对于图相关的任务，维度为[1,*]。
- data.position	节点位置矩阵（Node position matrix），维度为[num_nodes, num_dimensions]。

- [Learning to Identify High Betweenness Centrality Nodes from
Scratch: A Novel Graph Neural Network Approach](https://arxiv.org/pdf/1905.10418.pdf)
- node initial feature = [$(d_v), 1, 1]

In [9]:
synthetic = []
between = []
for f in os.listdir(data_synthetic):
    if "score" in f:
        # ground truth of betweenness centrality
        p = os.path.join(data_synthetic, f)
        between.append(p)
    else:
        p = os.path.join(data_synthetic, f)
        synthetic.append(p)

between.sort()
synthetic.sort()

In [112]:
data_list = []

for index, f in enumerate(synthetic):
    edge_index = torch_geometric.io.read_txt_array(f, dtype=torch.long)
    edge_index = edge_index.t().contiguous()
    edge_index = utils.to_undirected(edge_index)

    row, col = edge_index  
    deg = utils.degree(col) # must use col to get degree, why?
    deg = deg.numpy()  

    vertice = []
    for d in deg:
        vertice.append([d, 1, 1])
    vertice = np.array(vertice, dtype=np.float)
    vertice = torch.from_numpy(vertice)
    
    ### between centrality
    bcs = []
    bc = torch_geometric.io.read_txt_array(between[index], dtype=torch.long)
    bc = bc.t().contiguous()
    row, col = bc
    bc = col
    bc = bc.numpy()
    for b in bc:
        bcs.append([b])

    data = Data(x=vertice, edge_index=edge_index, y=bcs)
    
    data_list.append(data)

loader = DataLoader(data_list, batch_size=2)
# print(loader)

# Model

In [113]:
from torch_geometric.nn import MessagePassing
import torch.nn.functional as F
from torch_geometric.nn import global_max_pool
from torch_geometric.typing import Adj, OptTensor
from torch_geometric.transforms import Distance

In [114]:
class Net(MessagePassing):
    def __init__(self, c, p, q, num_layers, aggr="add"):
        super(Net, self).__init__(aggr=aggr)
        
        self.num_layers = num_layers
        self.w_0 = torch.nn.Linear(in_features=c, out_features=p).double()
        
        self.rnn = torch.nn.GRUCell(p, p).double()
  
        self.w_4 = torch.nn.Linear(in_features=p, out_features=q).double()
        self.w_5 = torch.nn.Linear(in_features=q, out_features=1).double()

    def forward(self, data):
        
        x, edge_index = data.x, data.edge_index
        
        # h_0 = x

        # h_1
        x = self.w_0(x)
        x = F.normalize(x, p=2, dim=1)
        
        row, col = edge_index
        deg = utils.degree(col, x.size(0), dtype=x.dtype)
        deg = torch.add(deg, 1)
        deg_inv_sqrt = torch.pow(deg, -0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        h_s = [x]
        
        
        for i in range(self.num_layers-1):
            # internally calls the message(), aggregate() and update() functions
            m = self.propagate(edge_index, x=x, norm=norm)
            x = self.rnn(m, x)
            x = F.normalize(x, p=2, dim=1) 
           
            h_s.append(x)
        
        h_s = torch.stack(h_s)
        z = global_max_pool(h_s, torch.tensor([0], dtype=torch.long))
        
        ### Decoder
        z = self.w_4(z)
        z = F.relu(z)
        z = self.w_5(z)
        
        return ret

    def message(self, x_j, norm: OptTensor):
        return x_j if norm is None else norm.view(-1, 1) * x_j
    

## Params

In [115]:
depth = 5
p = 128 # embedding dimension of hidden state
q = int(p/2)

epochs = 100
model_save = os.path.join(_root, "weight.pth")

In [116]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:1


In [117]:
model = Net(c=3, p=p, q=q, num_layers=depth).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-4)

criterion = torch.nn.BCEWithLogitsLoss()

In [118]:
for epoch in range(epochs):
    
    bce_loss = 0.0
    graph_cnt = 0
    for data in tqdm(loader):
        
        optimizer.zero_grad()
        
        data = data.to(device)
        bc_pr = model(data)
        bc_gt = data.y
        
        # row: source, col: det
        row, col = data.edge_index
        y_gt = bc_gt[col] - bc_gt[row]
        y_pr = bc_pr[col] - bc_gt[row]
        loss = criterion(y_pr, y_gt)
        
        bce_loss += data.num_graphs * loss.item()
        graph_cnt += data.num_graphs
        
        loss.backward()
        optimizer.step()
        
    print("Epoch = {}, loss = {}".format(epoch, bce_loss/graph_cnt))
    
    
    checkpoint = {
        'model_stat': model.state_dict(),
        'optimizer_stat': optimizer.state_dict(),
    }
    torch.save(checkpoint, model_save)

  0%|          | 0/15 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 80.00 MiB (GPU 1; 22.17 GiB total capacity; 535.20 MiB already allocated; 46.81 MiB free; 630.00 MiB reserved in total by PyTorch)